In [79]:
from selenium import webdriver
from PIL import Image
from io import BytesIO
import pytesseract

## Go to page

In [90]:
driver = webdriver.Chrome(executable_path = "../scraper/driver/chromedriver")
driver.get("https://www.shipmentlink.com/servlet/TDB1_CargoTracking.do")

In [18]:
language = driver.find_elements_by_css_selector("#shipmentlink_lang_layer button")[1]
language.click()

In [91]:
def set_attribute(driver, element, attribute, value):
    driver.execute_script("arguments[0].setAttribute(arguments[1], arguments[2]);", element, attribute, value)

In [92]:
imgelem = driver.find_element_by_id("captchaImg")
set_attribute(driver, imgelem, "width", None)
set_attribute(driver, imgelem, "height", None)

## Process

In [72]:
def black_and_white(image, pivot = 64, foreground = 0, background = 255):
    return Image.eval(image, lambda pixel: foreground if pixel <= pivot else background)

In [97]:
def process_image(image):
    string = pytesseract.image_to_string(image, config = "-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 7")
    return string.replace(" ", "")

In [98]:
imgdata = imgelem.screenshot_as_png
imgfile = Image.open(BytesIO(imgdata))
imgfile = black_and_white(imgfile, 64)
imgstr  = process_image(imgfile)
print(imgstr)

WSWH


In [88]:
captcha_input = driver.find_elements_by_id("captcha_input")[0]
captcha_input.clear()
captcha_input.send_keys(imgstr)

In [99]:
driver.close()

## Tests

In [10]:
a = {"type": "id", "selector": "etcetera", "commands": [{}, {}, {}], "required": True, "value": 234.56}

In [13]:
def print_command(command):
    command_texts = []
    for key, value in command.items():
        if isinstance(value, list):
            command_texts.append('"{0}": list({1})'.format(key, len(value)))
        elif isinstance(value, dict):
            command_texts.append('"{0}": dict({1})'.format(key, len(value)))
        elif isinstance(value, str):
            command_texts.append('"{0}": "{1}"'.format(key, value))
        else:
            command_texts.append('"{0}": {1}'.format(key, value))
    return "{" + ", ".join(command_texts) + "}"

In [14]:
print_command(a)

'{"type": "id", "value": 234.56, "required": True, "commands": list(3), "selector": "etcetera"}'